In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

ParserError: Error tokenizing data. C error: Expected 5 fields in line 3, saw 6


In [10]:

data.to_sql('boston', con, index=False, if_exists='replace')

506

In [11]:
pd.read_sql(
    '''
    SELECT * FROM boston
    --limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [27]:
pd.read_sql(
    '''
    SELECT 
        count(*) - count (CRIM) as CRIM_cnt_null,
        count(*) - count (ZN) as ZN_cnt_null,
        count(*) - count (INDUS) as INDUS_cnt_null,
        count(*) - count (CHAS) as CHAS_cnt_null,
        count(*) - count (NOX) as NOX_cnt_null
        
    FROM boston
    
    ''',
    con,
)

,CRIM_cnt_null,ZN_cnt_null,INDUS_cnt_null,CHAS_cnt_null,NOX_cnt_null
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [30]:
pd.read_sql(
    '''
    SELECT 
        count (DISTINCT CRIM) as CRIM_cnt,
        count (DISTINCT ZN) as ZN_cnt,
        count (DISTINCT INDUS) as INDUS_cnt,
        count (DISTINCT CHAS) as CHAS_cnt,
        count (DISTINCT NOX) as NOX_cnt
        
    FROM boston
    
    ''',
    con,
)

,CRIM_cnt,ZN_cnt,INDUS_cnt,CHAS_cnt,NOX_cnt
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.

In [52]:
pd.read_sql(
    '''
    SELECT *
        
    FROM (
    SELECT 'Med_CRIM' AS column, Med_CRIM AS value, (select min(CRIM) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_ZN' AS column, Med_ZN AS value, (select min(ZN) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_INDUS' AS column, Med_INDUS AS value, (select min(INDUS) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_CHAS' AS column, Med_CHAS AS value, (select min(CHAS) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_NOX' AS column, Med_NOX AS value, (select min(NOX) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1)))
    WHERE value = minsd
    
    ''',
    con,
)


,column,value,minsd
0,Med_CHAS,0.0,0.0


объяснение того, что медиана схожа с мин.значением в том, что мало мест с границами у реки и среди большого числа без реки и выпала медиана.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами.

Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.